<a href="https://colab.research.google.com/github/bryaanabraham/Wandering_Guardian/blob/main/ML%20Implementation/ML_Navigation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [367]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import folium
import random
from geopy.distance import geodesic

In [368]:
coords = pd.read_csv("training_dataset.csv")
coords.head()

,longitude,latitude,path_id
0,12.821062,80.044729,1.0
1,12.821219,80.044743,1.0
2,12.821479,80.044736,1.0
3,12.821561,80.044743,1.0
4,12.821636,80.044743,1.0


In [369]:
path = []
for i in range(1,len(coords['path_id'].unique())):
    path.append(coords[coords['path_id'] == i])

In [370]:
path[0].head()

,longitude,latitude,path_id
0,12.821062,80.044729,1.0
1,12.821219,80.044743,1.0
2,12.821479,80.044736,1.0
3,12.821561,80.044743,1.0
4,12.821636,80.044743,1.0


In [371]:
# Deviation parameters
min_deviation_distance = -0.00004
max_deviation_distance = 0.00004
#deviation is set to zero for now so elevation remains constant
max_deviation_angle = 0

# Generate random deviation distances and angles for each point in the DataFrame
num_points = len(coords)
random_deviation_distances = np.random.uniform(min_deviation_distance, max_deviation_distance, num_points)
random_deviation_angles = np.random.uniform(0, max_deviation_angle, num_points)

# Convert random angles to radians
random_deviation_angles_radians = np.radians(random_deviation_angles)

# Calculate deviation in latitude and longitudet
deviation_distance_lat = random_deviation_distances * np.cos(random_deviation_angles_radians)
deviation_distance_lon = random_deviation_distances * np.sin(random_deviation_angles_radians)

decoy_coords = coords.copy()
decoy_coords['latitude'] += deviation_distance_lat
decoy_coords['longitude'] += deviation_distance_lon

decoy_coords.head()

,longitude,latitude,path_id
0,12.821062,80.044744,1.0
1,12.821219,80.044764,1.0
2,12.821479,80.044768,1.0
3,12.821561,80.044745,1.0
4,12.821636,80.044773,1.0


In [373]:
print('coords keys:', coords.keys())
print('decoy_coords keys:', decoy_coords.keys())

coords keys: Index(['longitude', 'latitude', 'path_id'], dtype='object')
decoy_coords keys: Index(['longitude', 'latitude', 'path_id'], dtype='object')


In [374]:
decoy_path = []
for i in range(1, len(decoy_coords['path_id'].unique())):
    decoy_path.append(decoy_coords[decoy_coords['path_id'] == i])

decoy_path[0].head()

,longitude,latitude,path_id
0,12.821062,80.044744,1.0
1,12.821219,80.044764,1.0
2,12.821479,80.044768,1.0
3,12.821561,80.044745,1.0
4,12.821636,80.044773,1.0


In [375]:
maps = []

max_lat = max(coords['longitude'])
min_lat = min(coords['longitude'])
min_lon = min(coords['latitude'])
max_lon = max(coords['latitude'])

n = len(coords['path_id'].unique())-1

# each path_id has its own map
for i in range(0,n):

  m = folium.Map()
  gps_path = list(zip(path[i]['longitude'], path[i]['latitude']))
  folium.PolyLine(locations=gps_path, color='blue', weight=2.5, opacity=0.8).add_to(m)
  folium.Marker(gps_path[20], icon=folium.Icon(color='blue'), popup=f'Safe Path').add_to(m)

  gps_decoy_path = list(zip(decoy_path[i]['longitude'], decoy_path[i]['latitude']))
  folium.PolyLine(locations=gps_decoy_path, color='red', weight=2.5, opacity=0.8).add_to(m)
  folium.Marker(gps_decoy_path[10], icon=folium.Icon(color='red'), popup=f'Decoy Path').add_to(m)

  m.fit_bounds([(min_lat, min_lon), (max_lat, max_lon)])
  maps.append(m)

  display(maps[i])


In [376]:
def distance(current_point, path):
  nearest_point_on_line = min(path, key=lambda point: geodesic(point, current_point).meters)
  return geodesic(current_point, nearest_point_on_line).meters

In [377]:
def assign_color(distance):
  if distance < 2:
      return 'green'
  elif distance >= 4:
      return 'red'
  else:
      return 'orange'

In [378]:
maps = []

max_lat = max(coords['longitude'])
min_lat = min(coords['longitude'])
min_lon = min(coords['latitude'])
max_lon = max(coords['latitude'])

n = len(coords['path_id'].unique())-1

for i in range(n):
    m = folium.Map()
    gps_path = list(zip(path[i]['longitude'], path[i]['latitude']))
    folium.PolyLine(locations=gps_path, color='blue', weight=2.5, opacity=0.8).add_to(m)

    gps_decoy_path = list(zip(decoy_path[i]['longitude'], decoy_path[i]['latitude']))
    folium.PolyLine(locations=gps_decoy_path, color='red', weight=2.5, opacity=0.8).add_to(m)

    for index, row in decoy_path[i].iloc[::3].iterrows():
        current_point = (row['longitude'], row['latitude'])
        perp_dist = distance(current_point, gps_path)
        color = assign_color(perp_dist)
        folium.Marker(location=current_point,
                      popup=f"Deviation: {perp_dist:.2f} m",
                      icon=folium.Icon(color=color)).add_to(m)

    m.fit_bounds([(min_lat, min_lon), (max_lat, max_lon)])
    maps.append(m)

for map_obj in maps:
    display(map_obj)